In [ ]:
# Andrew Dunn, Katherine Dumais, Kathryn Link-Oberstar, Lee-Or Bentovim
# Summary of initial feature engineering and model testing for checkpoint 2

# This analysis runs after we load, clean, and generate additional variables in load_data.py, 
# and initially explored the data in exploratory_analysis.ipynb.

In [ ]:
# Import libraries and data

**Feature Engineering**



**Model Testing**

We tested variations of four different models: random forest, naive bayes, KNN, and logistic regression in the following notebooks:

Below, we demonstrate the best performing variants of those models.

In [ ]:
# Random Forest

In [ ]:
# Naive Bayes

In [ ]:
# KNN

In [ ]:
# Logistic Regression

** Discussion **